## Домашная работа №1. 

*Екатерина Волошина*

In [18]:
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import random
import time
from collections import Counter
from sklearn.model_selection import train_test_split

In [2]:
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

In [3]:
from nltk.tokenize import WordPunctTokenizer
from pymorphy2 import MorphAnalyzer

WPT = WordPunctTokenizer()
morph_parse = MorphAnalyzer()

In [4]:
session = requests.session()

Функция находит ссылки на фильмы на странице Афиши:

In [5]:
def find_movies():
    url = 'https://www.afisha.ru/msk/schedule_cinema/vybor-afishi/'
    req = session.get(url, headers={'User-Agent':ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    lists = soup.find_all('h3', {'class':'heHLK'})
    movies = []
    for i in lists:
        movie = i.find('a')['href']
        movies.append(movie)
    return movies

Функция достает позитивные отзывы про один фильм:

In [6]:
def download_positive_reviews(page):
    url = f'https://afisha.ru{page}reviews/positive'
    for _ in range(5):
        response = session.get(url)
        time.sleep(random.uniform(1.1, 5.2))

    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    reviews = soup.find_all('div', class_ ='restrict-text review__text')
    posts = [post.text for post in reviews]
    return posts

Функция достает отрицательные отзывы про один фильм:

In [7]:
def download_negative_reviews(page):
    url = f'https://afisha.ru{page}reviews/negative'
    for _ in range(5):
        response = session.get(url)
        time.sleep(random.uniform(1.1, 5.2))
    page = response.text
    soup = BeautifulSoup(page, 'html.parser')
    reviews = soup.find_all('div', class_ ='restrict-text review__text')
    posts = [post.text for post in reviews]
    return posts 

Функция разбивает выборку на тренировочную и тестовую и создает словарь с правильными ответами для тестовой выборки:

In [20]:
def reviews_split(pos, neg):
    pos_train, pos_test = train_test_split(pos)
    neg_train, neg_test = train_test_split(neg)
    test = {}
    for i in pos_test:
        test[i] = 1
    for i in neg_test:
        test[i] = 0
    return pos_train, neg_train, test

Функция создает множество слов, встречающихся только в положительных и только в отрицательных отзывах:

In [23]:
def analyze_reviews(pos, neg, freq):
    positive = []
    negative = []
    for i in pos:
        positive.extend(text_from_the_post(i))
    for i in neg:
        negative.extend(text_from_the_post(i))
    pos_words = Counter(positive)
    neg_words = Counter(negative) 
    positive = set(positive)
    negative = set(negative)
    common_words = positive.intersection(negative)
    positive -= common_words
    negative -= common_words
    positive = [i for i in positive if pos_words[i] > freq]
    negative = [i for i in negative if neg_words[i] > freq]
    return positive, negative

Функция лемматизирует текст:

In [10]:
def text_from_the_post(post):
    post = post[1:] # первый символ строки - оценка в десятибалльной системе
    text = WPT.tokenize(post.lower())
    words = [morph_parse.parse(i)[0].normal_form for i in text if i.isalpha()]
    return words

Функция определяет тональность в зависимости от того, каких слов в отзыве больше - слов, которые встречаются только в положительных или только в отрицательных отзывах. Нейтральные (если одинаковое количество положительных и отрицательных слов) отзывы будем трактовать как отрицательные

In [15]:
def tonality(post, pos, neg):
    text = text_from_the_post(post)
    pos_count = 0
    neg_count = 0
    mark = 0
    for i in text:
        if i in pos:
            pos_count += 1
    if pos_count > neg_count:
          mark = 1
    elif neg_count > pos_count:
          mark = 0
    return mark

Находим фильмы:

In [12]:
movies = find_movies()

Скачиваем отзывы:

In [13]:
p_words = []
n_words = []
for page in tqdm(movies):
    positive_words = download_positive_reviews(page)
    negative_words = download_negative_reviews(page)
    p_words.extend(positive_words)
    n_words.extend(negative_words)

Разделяем выборку:

In [21]:
p_train, n_train, test = reviews_split(p_words, n_words)

Создаем множества слов, которые встречаются только в положительных и только в отрицательных отзывах:

In [34]:
pos_words, neg_words = analyze_reviews(p_train, n_train, 5)

Определяем тональность на тестовой выборке и считаем accuracy: 

In [ ]:
accuracy = 0
for k, v in test.items():
    mark = tonality(k, pos_words, neg_words)
    if mark == v:
        accuracy += 1
accuracy = accuracy/len(test)

Смотрим, какая выходит accuracy:

In [36]:
accuracy

0.5769230769230769

#### Возможные улучшения:

1. Использовать tf-idf, чтобы оценить важность слов для определения отзыва, и обучить логистическую регрессию
2. Смотреть на н-граммы, а не просто слова (учитывать более широкий контекст)
3. Значительно увеличить выборку
4. Использовать другие лемматизаторы